In [1]:
!runSModelS.py -V

3.0.0-beta


In [2]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('/home/yoxara/smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.experiment.databaseObj import Database
from smodels.base.model import Model
from smodels.decomposition import decomposer
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.base import runtime
from smodels.experiment.expSMS import ExpSMS
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
#runtime.modelFile = 'smodels.share.models.ppZpjj'

/tmp/ipykernel_8395/2356514541.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import glob,imp


#### Observed

In [3]:
BSMlist = getParticlesFromSLHA('/home/yoxara/2MDM/Validated_Data/slhaFiles/xsec_ppZp_CMS_Obs/run_18_tag_1.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
#model.updateParticles(inputFile=slhaFile)
print(BSMlist)

[xd, xd~, y1]


In [4]:
!runSModelS.py -p /home/yoxara/2MDM/Validated_Data/parameters_simp.ini -f /home/yoxara/2MDM/Validated_Data/slhaFiles/xsec_ppZp_CMS_Obs -o /home/yoxara/2MDM/Validated_Data/results/Simp_output_Obs

INFO in databaseObj.loadBinaryFile() in 526: loading binary db file /home/yoxara/smodels/smodels-database/db3.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 533: Loaded database from /home/yoxara/smodels/smodels-database/db3.pcl in 2.5 secs.
INFO in modelTester.loadDatabaseResults() in 498: Including non-validated results
INFO in modelTester.testPoints() in 358: Running SModelS for 31 files with a single process. Messages will be redirected to smodels.log


In [5]:
slhaFolder = '/home/yoxara/2MDM/Validated_Data/slhaFiles/xsec_ppZp_CMS_Obs'
resultsFolder = '/home/yoxara/2MDM/Validated_Data/results/Simp_output_Obs'

In [6]:
path = "/home/yoxara/smodels/smodels-database" # Database path and SLHA file
slhaFile = "/home/yoxara/2MDM/Validated_Data/slhaFiles/xsec_ppZp_CMS_Exp/run_18_tag_1.slha"
db = Database(path) # Load database and model
analysis_list = ['CMS-EXO-19-012']
db.selectExpResults(analysisIDs=analysis_list, dataTypes='upperLimit', txnames='TRV1qq', useNonValidated=True)
print(db)

INFO in databaseObj.loadBinaryFile() in 526: loading binary db file /home/yoxara/smodels/smodels-database/db3.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 533: Loaded database from /home/yoxara/smodels/smodels-database/db3.pcl in 2.5 secs.


Database version: 3.0.0-beta
-----------------------------
1 experimental results: 1 CMS, 0 ATLAS, 0 @ 8 TeV, 1 @ 13 TeV
1 datasets, 1 txnames.



In [7]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)
    data.append(dataDict)

In [8]:
print(len(data))

32


In [9]:
#Convert data to flat DataFrame:
smodelsDF = json_normalize(data)
smodelsDF

,filename,SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,...,ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_24_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",10.088530,0.0,21.92031,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 4100.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.539763,0.850943,"[(Y1, -1)]"
1,run_25_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",7.174426,0.0,15.06892,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 4200.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.523893,0.575241,"[(Y1, -1)]"
2,run_17_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",10.950040,0.0,18.26174,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 3400.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.400384,0.458883,"[(Y1, -1)]"
3,run_02_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",121.069200,0.0,169.55050,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 1900.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.285941,0.466408,"[(Y1, -1)]"
4,run_01_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",57.872780,0.0,76.26473,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 1800.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.241159,0.307234,"[(Y1, -1)]"
5,run_10_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",32.289940,0.0,49.52041,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 2700.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.347947,0.544015,"[(Y1, -1)]"
6,run_13_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",28.015760,0.0,48.10580,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 3000.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.417622,0.878553,"[(Y1, -1)]"
7,run_06_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",14.181720,0.0,16.90138,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 2300.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.160913,0.051793,"[(Y1, -1)]"
8,run_27_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",6.787755,0.0,14.95754,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 4400.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.546200,0.525145,"[(Y1, -1)]"
9,run_21_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",12.509320,0.0,25.59416,0.0,0.0,50,0.2,1,...,[TRV1qq],"[(Y1, 3800.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.511242,1.004055,"[(Y1, -1)]"


In [10]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 5000001])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 9000006])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [11]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [12]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [13]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names

Final number of data points: 34


In [14]:
#Save DataFrame to pickle file:
dataDF.to_pickle('/home/yoxara/2MDM/Validated_Data/results/Simp_scanResults_Obs.pcl')
print(len(dataDF))
dataDF

34


,filename,mass.5000001,mass.5000521,extpar.1,extpar.2,extpar.3,extpar.4,xsec13TeV(fb).5000001,"BRs.y1.xd~,xd","BRs.y1.q,q",...,ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_24_tag_1.slha,4100.0,1.0,0.0,1.0,0.0,0.48610,21.920306,0.190358,0.134941,...,[TRV1qq],"[(Y1, 4100.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.539763,0.850943,"[(Y1, -1)]"
1,run_25_tag_1.slha,4200.0,1.0,0.0,1.0,0.0,0.45150,15.068924,0.214163,0.130973,...,[TRV1qq],"[(Y1, 4200.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.523893,0.575241,"[(Y1, -1)]"
2,run_17_tag_1.slha,3400.0,1.0,0.0,1.0,0.0,0.28902,18.261738,0.399429,0.100096,...,[TRV1qq],"[(Y1, 3400.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.400384,0.458883,"[(Y1, -1)]"
3,run_02_tag_1.slha,1900.0,1.0,0.0,1.0,0.0,0.20426,169.550470,0.571119,0.071485,...,[TRV1qq],"[(Y1, 1900.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.285941,0.466408,"[(Y1, -1)]"
4,run_01_tag_1.slha,1800.0,1.0,0.0,1.0,0.0,0.17744,76.264734,0.638292,0.060290,...,[TRV1qq],"[(Y1, 1800.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.241159,0.307234,"[(Y1, -1)]"
5,run_10_tag_1.slha,2700.0,1.0,0.0,1.0,0.0,0.24627,49.520411,0.478088,0.086987,...,[TRV1qq],"[(Y1, 2700.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.347947,0.544015,"[(Y1, -1)]"
6,run_13_tag_1.slha,3000.0,1.0,0.0,1.0,0.0,0.30522,48.105811,0.373574,0.104405,...,[TRV1qq],"[(Y1, 3000.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.417622,0.878553,"[(Y1, -1)]"
7,run_06_tag_1.slha,2300.0,1.0,0.0,1.0,0.0,0.13295,16.901384,0.758637,0.040228,...,[TRV1qq],"[(Y1, 2300.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.160913,0.051793,"[(Y1, -1)]"
8,run_27_tag_1.slha,4400.0,1.0,0.0,1.0,0.0,0.50188,14.957533,0.180705,0.136550,...,[TRV1qq],"[(Y1, 4400.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.546200,0.525145,"[(Y1, -1)]"
9,run_21_tag_1.slha,3800.0,1.0,0.0,1.0,0.0,0.42748,25.594156,0.233139,0.127811,...,[TRV1qq],"[(Y1, 3800.0)]",CMS-EXO-19-012,None,13.0,137.0,upperLimit,0.511242,1.004055,"[(Y1, -1)]"
